In [22]:
from pymongo import MongoClient
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import numpy as np
init_notebook_mode(connected=True)
BOOKS_DB = 'bookmate'

In [17]:
def connect_to_mongo_database(db):
    client = MongoClient('localhost', 27017)
    db = client[db]
    return db

In [18]:
def plot_data(dbs, book_id, field):
    plot_data = []
    for db in dbs:
        database = connect_to_mongo_database(db)
        data = database['%s_pages' % book_id].find().distinct(field)
        data = np.convolve(data, np.ones(10)/10)
        trace = go.Scatter(
            x = list(range(0, len(data))),
            y = data,
            mode = 'lines',
            name = db.title()
        )
        plot_data.append(trace)

#     iplot(plot_data, filename='%s_%s.png' % (book_id, field))

In [ ]:
book_ids = ['2206', '2207', '2543', '2289', '135089']
fields = ['page_speed', 'page_skip_percent', 'page_unusual_percent']
# fields = ['page_speed']
# book_ids = ['2206']
dbs = ['bookmate', 'bookmate_male', 'bookmate_female', 'bookmate_paid', 'bookmate_free']

for book_id in book_ids:
    for field in fields:
        plot_data(dbs, book_id, field)

In [19]:
def get_users_info(sessions_collection, book_id):
    db = connect_to_mongo_database(BOOKS_DB)
    users = db[sessions_collection].find().distinct('user_id')
    
    sessions = []
    for user_id in users:
        user_sessions = db[sessions_collection].find({'user_id': user_id})
        db['%s_users' % book_id].update({'_id': user_id},
                                        {'$set': {'sessions': user_sessions.count()}})
        sessions.append(user_sessions.count())
    data = [go.Histogram(x=sessions)]
    iplot(data)

In [ ]:
def get_users_in_range(sessions_collection, book_id, user_id, min_sessions, max_sessions):
    db = connect_to_mongo_database(BOOKS_DB)
    sessions = db[sessions_collection].find({'user_id':user})

In [ ]:
book_id = '2206'
sessions_collection = '%s_users' % book_id
get_users_info(book_id, sessions_collection)